In [1]:
import os

os.chdir("../")

%pwd

'c:\\Projects\\DSML\\MachineLearning-01'

In [3]:
import pandas as pd

data = pd.read_csv("artifacts\data_ingestion\winequality-red.csv")

data.head(10)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [21]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__ (
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema
            )
        
        return data_validation_config

In [22]:
import os
import pandas as pd
from mlProject import logger

class DataValidation:
    def __init__ (self, config = DataValidationConfig):
        self.config = config

    def validate_all_columns (self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    
                else:
                    validation_status = True

                with open(self.config.STATUS_FILE, "a") as f:
                        f.write(f"validation status {col}: {validation_status}\n")
            return validation_status

        except Exception as e:
            raise e


In [23]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()

except Exception as e:
    raise e
    

2025-11-20 14:22:40,517 : INFO : common : YAML file config\config.yaml read successfully.
2025-11-20 14:22:40,522 : INFO : common : YAML file params.yaml read successfully.
2025-11-20 14:22:40,528 : INFO : common : YAML file schema.yaml read successfully.
2025-11-20 14:22:40,535 : INFO : common : created directory at artifacts
2025-11-20 14:22:40,536 : INFO : common : created directory at artifacts/data_validation
